In [ ]:
# LIBRARIES
# Data Wrangling
import pandas as pd
import numpy as np
from collections import Counter
import re
import math
import joblib
import io
import base64
import pickle
from time import sleep
import random
import string
# Visualization
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode()
import plotly.express as px
from IPython.display import display
# Spatial
import folium
from folium import Map,Choropleth, Circle, Marker,LayerControl,GeoJson,Icon,Popup
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import IFrame
import branca.colormap as branca_folium_cm
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon
# Temporal
from datetime import datetime,timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
# Text
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
stopwords_nltk =nltk.corpus.stopwords.words('spanish')
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk.util import ngrams
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from textblob import Word,TextBlob
!pip install langdetect
from langdetect import detect
# Modeling
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
!python -m spacy download es_core_news_sm
from spacy.lang.es import Spanish
import es_core_news_sm
nlp = es_core_news_sm.load()
# Modeling
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_curve,roc_auc_score,log_loss,accuracy_score,silhouette_samples,silhouette_score,plot_confusion_matrix,classification_report,confusion_matrix

# Sentiment and GeoSpatial Analysis in Peruvian Food Reviews

Natural language processing (NLP) refers to the branch of artificial intelligence concerned with giving computers the ability to understand text and spoken words in much the same way human beings can. On the other hand, Geospatial analysis is the gathering, display, and manipulation of imagery, GPS, satellite photography and historical data, described explicitly in terms of geographic coordinates or implicitly, in terms of a street address, postal code, or forest stand identifier as they are applied to geographic models.

About the topic, Peru is one of best and peculiar culinary destination in the world. This country has diverse climates and ecological floors, where various crops have been developed. In this way, it has a lot of natural and unique inputs. So, peruvian food is a cuisine of opposites: hot and cold on the same plate. Acidic tastes melding with the starchy. Robust and delicate at the same time. All with an excellent result.

About the Kaggle dataset, [**Peruvian Food Reviews**](https://www.kaggle.com/lazaro97/peruvian-food-reviews) have a diversity of information. This have text, spatial and temporal features! It's a good opportunity to show some mixed ideas about sentiment analysis, geospatial analysis and time series analysis. Possible uses here are: market research, create an guide app for clients.

<center><img src="https://th.bing.com/th/id/OIP.cs5iCAa15Cz-27OnLnY6LgHaDl?pid=ImgDet&rs=1" alt="centered image" height="898" width="678"> </center>

In general, the project that i did are the following steps:
- Extraction of Web-Data: Using Web Scraping in the platforms TripAdvisor and GoogleMaps.
- Preprocessing: The computer should read all features. It exists a set of steps there.
- Exploratory Analysis: Some first plots and get a first idea of the data.
- Clustering: A review can be classified by emotion or sentiment. That is possible using Spanish NrcLexicon.
- Classification: Identify what reviews are good, bad or so-so, and what is the probability to occur that.
- Reports: The restaurants (or districts) have unique features and is interesting to see the diversity of patterns in each one.
- Dashboard: Summary of all project where it exists interactivity with an user.
<div class="alert alert-block alert-success">  
Hello everybody! 😊<br>
I have some lines of code (Python, R) in my local computer. If you like this series of notebooks (and dataset) don't forget to upvote. That's a good motivation to continue in create similar projects in near future.<br>
The way that i got the dataset you can find <a href=https://github.com/Lazaro-97/satisfaction-in-restaurants>here</a>. If you are interested in a specify country, you can change and adapt the code.<br>
Pd: Please see the dictionary of data, that contains a lot of information. At the moment this is the <B>base notebook</B>, i hope to see more complex works 😊.
</div>  

# Preprocessing

In [ ]:
# GET DATA
# General data
reviews_df=pd.read_csv('../input/peruvian-food-reviews/data/data/reviews.csv')
reviews_df.dropna(subset=['review'],inplace=True)
reviews_df.drop_duplicates(subset=['review'],inplace=True,keep=False)
restaurants_df=pd.read_csv('../input/peruvian-food-reviews/data/data/restaurants.csv')
# Auxiliar data
borders=gpd.read_file('../input/peruvian-food-reviews/data/data/geospatial/districts.shp')
nrc=pd.read_csv('../input/peruvian-food-reviews/data/data/spanish-nrc/lexico_nrc.csv')
# PREPROCESSING
trans=str.maketrans('áéíóúüñÁÉÍÓÚÜ','aeiouunAEIOUU')
lemma_n={'limar':'lima','increiblemente':'increible','normalito':'normal','encantado':'encantar','lamentablemente':'lamentable','buenazo':'bueno','buenisimo':'bueno','buenisima':'bueno','zoolog':'zoo','abuelit':'abuel','malisimo':'malo','decepcion':'decepcio','decepcionado':'decepcio','decepcioado':'decepcio','decepcioante':'decepcio','riquisimo':'rico','recomendado':'recome','recomendacion':'recome','recomeir':'recome','recomer':'recome','recomendar':'recome','recomendarr':'recome','recomer':'recome','recomendarrr':'recome','recomendable':'recome','deliciosa':'delicia','delicioso':'delicia','delicios':'delicia','pesima':'pesimo','fria':'frio','atencion':'atender','comida':'comer','ambientar':'ambiente','venezuela':'venezolano','preciar':'precio','malisima':'malo'}
list_stopwords=list(set(stopwords_nltk+list(STOP_WORDS)+['\n','\t','nan','','aa',' ','q','pq','and','xd','xq']))
def remove_translate(x):
    x=str(x)
    if x.__contains__('(Traducción de Google)'): return x[len('(Traducción de Google)')+1:x.find('(Original)')]
    else: return x
def create_lemma(x,dc):
    if any(ext in x for ext in dc.keys()): 
        for k in dc.keys():
            x=str(x).replace(k, str(dc[k]))
        return x.replace('recome','recomendar').replace('ceviche','cebiche')
    else: return x.replace('recome','recomendar').replace('ceviche','cebiche')
def preprocess_text(x):
    try:
        x=remove_translate(x)
        x=x.lower()
        x = re.sub('\[.*?¿\]\%', '', x)
        x = re.sub('[%s]' % re.escape(string.punctuation), '', x)
        x = re.sub('\w*\d\w*', '', x)
        x = re.sub('[‘’“”…«»]', '', x)
        x = re.sub('\n', ' ', x)
        x=x.replace('<br>','').replace('</br>','')
        x= [t.lemma_.lower().translate(trans) for t in nlp(x) if (not t.is_stop) &(t.text not in list_stopwords)]
        x=' '.join(x)
        return create_lemma(x,lemma_n)
    except Exception as e:  return ''
def get_entities(x):
	try: return ' '.join([token.text for token in nlp(x) if (token.ent_type!=0) & (token.is_stop==False)])
	except: return ''
def get_labels(x):
    if x>3: return 'Excellent'
    elif x<3: return 'Bad'
    else: return 'Ok'
def translate_lg(x):
	try: return detect(x)
	except: return ' '
reviews_df['language']=reviews_df['review'].apply(translate_lg) #Is not precise
reviews_df=reviews_df[reviews_df['language']=='es']
reviews_df['label']=reviews_df['score'].apply(lambda x: 'Excellent' if x>3 else 'Bad' if x<3 else 'Ok')
reviews_df['pre_review']=reviews_df['review'].apply(preprocess_text)
#reviews_df['entity']=reviews_df['review'].apply(get_entities) In 1rst and 2nd version
reviews_df['date']=pd.Categorical(reviews_df['date'],categories=['1 months ago','2 months ago','3 months ago','4 months ago','5 months ago','6 months ago','7 months ago','8 months ago','9 months ago','10 months ago','11 months ago','12 months ago','1 years ago','2 years ago','3 years ago','4 years ago','5 years ago','6 years ago','7 years ago','8 years ago','9 years ago','10 years ago','11 years ago','12 years ago','13 years ago'],ordered=True)
def get_tags(df):
	vectorizer=CountVectorizer(tokenizer = lambda x:[x.split('||', 1)[0].strip()])
	mat=vectorizer.fit_transform(df.tag).toarray()
	mat=pd.DataFrame(mat,columns=vectorizer.get_feature_names())
	mat.drop(['','$', '$$ - $$$', '$$$$'],axis=1,inplace = True)
	mat=pd.concat([df.id,mat],axis = 1)
	mat.to_csv('./tag.csv',index=False)
#get_tags(restaurants_df) In 1rst and 2nd version
all_df=reviews_df.merge(restaurants_df,how='left',left_on='service',right_on='id')[['id_review', 'review', 'title', 'score', 'likes', 'id_nick', 'service','date', 'label', 'pre_review', 'id', 'name', 'tag','x', 'y', 'district', 'IDDIST']]
all_df.to_csv('./all.csv')

Saving the infomation to use in another moment:
- All: Reviews with new **features**: preprocessed text, detection of entities, new label in scores, and **conexion** with Restaurants getting district, x, y and name of restaurant.
- Tag: Type of restaurant. What products do they sell?

# Geospatial Analysis

In [ ]:
def geo_map(df,dg,districts):
	districts = districts[["IDDIST", "geometry"]].set_index("IDDIST")
	map = Map(location=[-12.3101093,-76.8850579], tiles='Stamen Toner', zoom_start=9.2) 
	Choropleth(geo_data=districts.__geo_interface__,
          data=df['stars'],
          key_on="feature.id",
           fill_color='Blues',
         #  legend_name="Satisfaction of restaurant's services in Lima by district"
          ).add_to(map)
	HeatMap(data=zip(dg['x'], dg['y'],dg['n_reviews']), radius=30).add_to(map)
	mc = MarkerCluster()
	for idx, row in dg.iterrows():
		html=f"""
		    <h1 style="color:DodgerBlue;"> <b> {row['name']} </b></h1>
		    <p> {row['stars']/10} estrellas </p>
		    <p> {row['n_reviews']} reseñas </p>
		     <p> {row['direction']} </p>
		    """
		popup = Popup(folium.IFrame(html=html, width=500, height=300), max_width=2650)
		mc.add_child(Marker([row['x'], row['y']],color='red',popup = popup,icon=Icon(color='blue')))#
	map.add_child(mc)
	return embed_map(map,'./map.html')
def embed_map(m, file_name):
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')
dgs=restaurants_df[restaurants_df['IDDIST']!='Out']
dg=dgs.groupby(by="IDDIST").mean()
ds=borders[borders['PROVINCIA']=='LIMA']
geo_map(dg,dgs,ds)

- The heatmap and red scale represents the number of reviews.
- The choropleth map and blue scale represents the mean score by district.
- The ubication of the restaurants in markers.

Pd: The map can be adapted for other cases. For example: analyzing the progress of COVID-19 vaccination, or maybe, analyzing the taccidents in any state.

# Create reports

In [ ]:
class report:
	def __init__(self,x,c,ids):
		self.df=x
		self.element=c
		self.ids=ids
	def generate_all(self):
		for idx in self.ids: 
			print(f'Analyzing the {self.element} {idx}')
			df=self.df[self.df[self.element]==idx]
			self.get_score(df)
			self.generate_wordcloud(df,idx)
			self.get_reviews(df)
			self.get_timeseries(df)
	def get_reviews(self,df):
		fill_color = []; n = len(df)
		for i in range(n):
			if df.iloc[i]['label']=='Excellent':fill_color.append('rgb(102, 178, 255)')
			elif df.iloc[i]['label']=='Bad':fill_color.append("rgb(255, 102, 102)")
			else:fill_color.append('rgb(153, 255, 204)')
		fig = go.Figure(data=[
	        go.Table(columnorder = [1,2], columnwidth = [440,40],
	      header=dict(values=['<b>Reviews</b>', '<b>Score</b>'],
	        line_color='black', fill_color='black',
	        align='center',font=dict(color='white', size=12)),
	      cells=dict(values=[df.review, df.score],
	        line_color=['black']*2,fill_color=[fill_color,fill_color],
	        align='center', font=dict(color='black', size=11)))
	    ])        
		fig.show()
	def generate_wordcloud(self,df,save):
		wordcloud = WordCloud(width=1640, height=1200,stopwords=set(stopwords.words('spanish')).union(set(['pre_review','Length','dtype','object','comida','comer','rico','atender','restaurante','servicio','plato','excelente','recomendar','delicia','ambiente','sabor','pedir','precio','agradable','local','calidad'])),
	        max_font_size=500,scale=3,random_state=60).generate(" ".join(df['pre_review'])) #Removing very common words in restaurants
		wordcloud.recolor(random_state=1)
		plt.figure(figsize=(25, 21))
		plt.imshow(wordcloud)
		plt.axis('off')
		plt.savefig(f'./wordcloud_{save}.png')
		plt.show()
	def get_score(self,df):
		dg=pd.DataFrame(df['score'].value_counts())
		dg.sort_index(inplace=True)
		dg.columns=['Number of reviews']
		clrs=['red' if (x > dg['Number of reviews'].sum()/3) else 'lightcoral' for x in dg['Number of reviews'] ]
		fig = go.Figure(data=[go.Bar(
            x=['Stars_1','Stars_2','Stars_3','Stars_4','Stars_5'],
            y=dg['Number of reviews'],
            marker_color=clrs
        )])
		fig.show()
	def get_timeseries(self,df):
		dg=df.groupby(by="date").agg(['count', 'mean'])
		dg.sort_index(inplace=True,ascending=False)
		dg.dropna(inplace=True)
		fig=go.Figure([go.Scatter(x=dg['score']['mean'].index, y=dg['score']['mean'].values)])
		fig.update_layout(paper_bgcolor='rgba(0,0,0,1)',  plot_bgcolor='rgba(0,0,0,1)',font=dict(size=10, color='#ffffff'),)
		fig.show()      

# Analyzing districts

In [ ]:
report(all_df,'district',['MIRAFLORES','SANTIAGO DE SURCO']).generate_all()

# Analyzing restaurants

In [ ]:
report(reviews_df,'service',[242558.0,275258.0]).generate_all() # Name of restaurants: Mango, El restaurante venezolano

- Distribution of score
- Wordcloud
- List of reviews
- Evolution of mean score

By district, By restaurant

# Classification

In [ ]:
def model(c=1):
    return Pipeline([('vect', CountVectorizer(analyzer = "word",min_df=20,stop_words = stopwords_nltk)),
                     ('tfidf', TfidfTransformer(norm = 'l2',use_idf = True)),
                     ('model', LogisticRegression(solver = 'newton-cg',max_iter = 1000,tol=1e-4,C = c))])
def training_cv(X_train,y_train,mod):
    skf=StratifiedKFold(n_splits = 10,shuffle = True,random_state = 60)
    yv=np.zeros(len(X_train))
    yvs=np.zeros((len(X_train),5))
    fi=pd.DataFrame(columns=[f"W_Class_{k}" for k in range(5)])
    for fold,(idx_tr,idx_vl) in enumerate(skf.split(X_train,y_train)):
        X_tr,y_tr=X_train.iloc[idx_tr],y_train.iloc[idx_tr]
        X_vl,y_vl=X_train.iloc[idx_vl],y_train.iloc[idx_vl]
        model = mod.fit(X_tr,y_tr)
        fi = pd.DataFrame(model['vect'].get_feature_names(), columns = ["Features"])
        for k in range(5): fi[f"W_Class_{k}"] = +pow(math.e, model['model'].coef_[k])
        yv[idx_vl]=model.predict(X_vl)
        yvs[idx_vl]=model.predict_proba(X_vl)
    return yv,yvs,fi
def evaluation(yreal,y,ys):
    print(classification_report(yreal, y))
    print('AUC: ',roc_auc_score(yreal, ys, multi_class='ovo', average='weighted'))
    print('Accuracy: ',accuracy_score(yreal, y))
    print('Log_Loss: ',log_loss(yreal, ys))
    plt.figure(figsize=(6,5))
    sns.heatmap(pd.DataFrame(confusion_matrix(yreal, y)), annot=True, linewidths=.5, fmt="d")
    plt.savefig('./confusion_matrix.png')
    plt.show()
    col=['red','orange','yellow','blue','green']
    plt.figure(figsize=(7,6))
    for k in range(5): sns.kdeplot(x=ys[:,k],fill=True,color = col[k])
    plt.savefig('./distributions.png')
    plt.show()
def model_weights(df):
#    df["OddsRatio_4-3"] = df['W_Class_4']/df['W_Class_3']
    print(df)
    i,j=0,0
    fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(8,7))
    for k in range(5):
        df=df.sort_values(by = [f"W_Class_{k}"], ascending=False)
        plt.subplot(2,3,k+1)
        if j>2: i=1;j=0
        df[:10].plot.barh(x='Features', y=f"W_Class_{k}",color='blue',ax=axes[i,j])
        j=j+1
    plt.savefig(f'feat_importance.png')	
    plt.show()

In [ ]:
reviews_df.drop_duplicates(inplace=True,subset=['pre_review'])
reviews_df.reset_index(inplace=True)
yv,yvs,fi=training_cv(reviews_df['pre_review'],reviews_df['score'],model())
model_weights(fi)
evaluation(reviews_df['score'],yv,yvs)
dfx=pd.DataFrame({'stars_1':yvs[:,0],'stars_2':yvs[:,1],'stars_3':yvs[:,2],'stars_4':yvs[:,3],'stars_5':yvs[:,4]})
pd.concat([reviews_df['id_review'],dfx],axis=1).to_csv('./model.csv',index=False)

- Saving the predictions to use in another moment.
- The model is good but the metrics can be better. Trying another options as Convolutional or Recurrent Neural Networks.
- The data have problems with lemmatization. Exist more resources using English language but i hope to find better results.

<div class="alert alert-block alert-success">  
Work in progress..</div>

Future Tasks here: A convolutional neural network model